# 第一步
保存原始文件到text.srt
运行后，格式化的文件保存到clean.srt

In [1]:
import re
punc = "？。＂＃＄％＆＇＊＋，,－／：；＜＝＞＠［＼］＾＿｀｛｜｝｢｣〃「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."

def exclude(c: str):
  if (c.lower() >= "a" and c.lower() <= "z") or (c >= "0" and c <= "9"):
    return True
  return False

def clean(line: str) -> str:
  line = line.strip()
  line = re.sub(r"[%s]" %punc, " ", line)
  new_lines = []
  for l in line.split(" "):
    new_line = l.strip()
    if new_line:
      if new_lines and (len(new_lines[-1]) + len(new_line) <= 10 or exclude(new_lines[-1][-1]) or exclude(new_line[0])):
        new_lines[-1] += f" {new_line}"
      else:
        new_lines.append(new_line)
  return new_lines

with open("text.srt") as file:
    lines = []
    for line in file:
      if line.strip():
        lines.extend(clean(line))

with open("clean.srt", "w") as file:
  for line in lines:
    file.write(f"{line}\n")
    

# 第二步
切每句话起终点
- 按空格+回车：建立下一句新起点
- 按1+回车：退出
- 按回车：建立当前句的终点

In [2]:
with open("clean.srt") as file:
    lines = []
    for line in file:
        lines.append(line.strip())

records = []
import time
start = time.time()
index = 0
content = lines[index]

while index < len(lines):
  s = input("####: " + content)
  if s == " ":
    pass
  elif s == "1":
    break
  else:
    records.append([content, start, time.time()])
    index += 1
    content = lines[index]
  start = time.time()

# 第三步
设置a为起始秒数，运行后，字幕文件保存到final.srt

In [3]:
a = 0

count = 1
process_records = records[1:]
_start = records[0][2]

def get_duration(cur):
  hour = int(cur / 3600)
  min = int(cur / 60)
  sec = int(cur) % 60
  milsec = int(cur * 1000) % 1000
  hour = str(hour) if hour >= 10 else "0" + str(hour)
  min = str(min) if min >= 10 else "0" + str(min)
  sec = str(sec) if sec >= 10 else "0" + str(sec)
  milsec = str(milsec) if milsec >= 100 else ("0" + str(milsec) if milsec >= 10 else "00"+str(milsec))
  return f"{hour}:{min}:{sec},{milsec}"

new_rows = []

for line in process_records:
  content, start, end = line[0], line[1], line[2]
  new_rows.append(str(count))
  new_rows.append(f"{get_duration(start-_start + a)} --> {get_duration(end-_start + a)}")
  new_rows.append(content)
  new_rows.append("")
  count += 1
with open("final.srt", "w") as file:
  for line in new_rows:
    file.write(f"{line}\n")